In [4]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import urllib.request
import time
import os
duration = 1  # seconds
freq = 440  # Hz

browser = webdriver.Chrome("/usr/bin/chromedriver")
browser.get('https://kmplot.com/analysis/index.php?p=service&cancer=ovar')
time.sleep(2)
fg = open( 'gene_names_7-11-2019.txt' , 'r' )
for line in fg.readlines() :
    try:
        element=browser.find_element_by_name('captcha_code')
    except NoSuchElementException:
        elem_present = False
    else: 
        elem_present = True
        
    searchBar = browser.find_element_by_id('affyid')
    jetSet = browser.find_element_by_id('jetset')
    draw_plot = browser.find_element_by_id('start_button')
    kmplotter = browser.find_element_by_id('analysis')
    time.sleep(2)
    
    affyid = line.split("\n")[0]
    searchBar.send_keys(affyid)
    jetSet.click()
        
    if elem_present!=False:
        os.system('play -nq -t alsa synth {} sine {}'.format(duration, freq))
        input("Please enter Captcha and press enter...")
        time.sleep(2)
    else:
        time.sleep(1)

    draw_plot.click()
    try:
        invalid_key = browser.find_element_by_class_name('errormsg')
    except NoSuchElementException:
        inv_id = True
    else:
        inv_id = False
    
    if inv_id == False:
        print(line)
        f_inv = open( 'invalid genes_7-11-2019.txt' , 'a+' )
        f_inv.write(line)
        f_inv.close()
        continue
    else: 
        time.sleep(1)
        
    pdf = browser.find_element_by_xpath('//*[@id="content"]/p[2]/a')
    pdf_file = pdf.get_attribute('href')
    urllib.request.urlretrieve(pdf_file, filename="temp")
    time.sleep(5)
    
    from tika import parser
    raw = parser.from_file("temp")
    f = open("hr_value-7-11-19.txt","w+")
    f.write(raw['content'])
    f.close()
    time.sleep(5)

    f_r=open('hr_value-7-11-19.txt', "r+")
    lines=f_r.readlines()
    HR = lines[81]
    plog = lines[82]

    hr_val = HR.split(" ")[2]
    hr_range = "".join(HR.split(" ")[3:6]).split("\n")[0]
    plog_val = plog.split(" ")[3]

    val_file = open("hr_value_all_7-11-19.txt","a+")
    val_file.write(",".join([affyid, hr_val, hr_range, plog_val]))
    val_file.close()
    
    try:
        element=browser.find_element_by_id('analysis')
    except NoSuchElementException:
        elem_present = False
    else: 
        elem_present = True
        
    if elem_present == False:
        input("Press Enter to continue...")
    else:
        element=browser.find_element_by_id('analysis')
        element.click()
        time.sleep(3)
    
fg.close()
f.close()
browser.quit()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 77
